A Data Analysis project that aims to extract insights out of the dataset that is a available on Kaggle "Heart-Attack-Prediction in the united states" This project aims to go on a step-by-step data analysis sequence. It starts by the most convenient, practical, and logical process to start with on data science which is, data cleaning. The project goes on five phases of data cleaning, which are. Handling missing values, remove duplicates, fix data types, detect and treate outliers, and check for inconsistencies. The project is to be complete on different stages of analysis (Heart attack risk factors, predicting heart attack probability, demographic disparities, and other more) and as each stage is done and complete, the Notebook will be edited accordingly.

**Finally, please note that this is a data analysis project that is being analyazed by a data analyzer who has no medical experience or knowledge. Thus, this project should not be used for any medical consumption or treat it is only applicable for research-based purposes.**

In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing


In [ ]:
df = pd.read_csv("/kaggle/input/heart-attack-prediction-in-united-states/heart_attack_dataset.csv")
df.head()

Before staritng the data analysis of this data set "Heart-attack-prediction. We need to preform some data cleaning process so the data is perfect to be analyzed. Some of the steps we are going to preform on cleaning the dataset involves handling missing values, remove duplicates, fix data types, and some more that we will highlight throughout the process
Now lets get started

# Data Cleaning

In [ ]:
#Handling Missing Values

df.isnull().sum()

Even though, the dataset showing 0-missing values for all columns, there still potential data cleaning needed to ensure perfection. 

In [ ]:
#Here we will look for any duplicates
#getting rid of duplicates ensures no distort or bias on the result of the analysis

df.duplicated().sum()


No dubplicats on the dataset,

In [ ]:
#Veryfing the data types
df.dtypes

All columns are on the corrct data-type. Yet, some columns(e.g.,Gender, SrtokHistory, PreviousHeartAttack, etc) can still be/get grouped under a 'category' data type. 

In [ ]:
#We will convert some of the object data type columns in to a category data type columns

categorics_cols = ["Gender", "Smoker", "Diabetes", "Hypertension", "FamilyHistory",
   "PhysicalActivity", "AlcoholConsumption", "Diet", "StressLevel", 
   "Ethnicity", "Income", "EducationLevel", "Medication", "ChestPainType",
   "ECGResults", "ExerciseInducedAngina", "Slope", "Thalassemia", "PreviousHeartAttack",
   "StrokeHistory", "Residence", "EmploymentStatus", "MaritalStatus"] #A list of the columns that we want to change its data type to categorical
df[categorics_cols] = df[categorics_cols].astype("category")#changing the object data type columns into categorical columns
df[categorics_cols].dtypes

Our object-type columns are now a categorical type.
that will help us in many way in our analysis. Such as, dealing with categorical data types is more memory efficient because they store unique valus only once. As well as operations like grouping/filtering are faster. 

In [ ]:
#Next, we will ensure that all the numerical columns are type int64 or float64

numerical_cols = ["Age", "Cholesterol", "BloodPressure", "HeartRate", 
  "BMI", "MaxHeartRate", "ST_Depression", "NumberOfMajorVessels"]
df[numerical_cols].dtypes

In [ ]:
#Now we will ensure that no value is stored as string accidentally

all_numeric = all(df[col].astype(str).str.isnumeric().all() for col in numerical_cols)
all_numeric

Getting a False indicates that we at least have one value in "all_numeric" that is not pure numeric string.

In [ ]:
#We will troubleshoot and find the values which are causing the problem

for col in numerical_cols:
    if not df[col].astype(str).str.isnumeric().all():#Checkin if the values are numeric
        print(f"Column '{col}' has non-numeric values:", df[col].astype(str)[~df[col].astype(str).str.isnumeric()].unique())
        

The output shows that the two columns BMI and ST_Depression have values which are not numeric.
str.isnumeric() only accepts whole numbers without decimal point. However, these values are valid floating-point numbers, not non-numeric data.

In [ ]:
#converts the columns (BMI and ST_Depression) to float64

df["BMI"] = pd.to_numeric(df["BMI"], errors='coerce')
df["ST_Depression"] = pd.to_numeric(df["ST_Depression"], errors='coerce')

df["BMI"].dtypes, df["ST_Depression"].dtypes

In [ ]:
#Handele NaN Values
df["BMI"] = df["BMI"].fillna(df["BMI"].mean()).infer_objects(copy=False)
df["ST_Depression"] = df["ST_Depression"].fillna(df["ST_Depression"].mean()).infer_objects(copy=False)

len(df["BMI"].isna()), len(df["ST_Depression"].isna())

this indicats that our two columns BMI and ST_Depression has no missing values

Next step after Fix Data Types, is to Detect and Treat Outliers.
Outliers are extrem values (e.g., Age=150). the IQR (InterQuartile Range) is a statical method to identify them. 
We will preform the IQR on all the numerical data type columns.

In [ ]:
#Calculating IQR for numerical type columns
#We will define a column that caluculate the IQR for us

def calculate_iqr_bounds(df, columns):
     results = {}
     for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        results[col] = {
            f"{col}: Lower bound = {lower_bound:.2f}. Values below this may be outliers.",
            f"{col}: Upper bound = {upper_bound:.2f}. Values above this may be outliers."
        }
     return results

This is a function that will not only help us calculating the IQR, but it will also print out the upper and lower ranges for the column.
Next we will us the function with all the numerical columns. Then, we will treat Outliers

In [ ]:
calculate_iqr_bounds(df, 
['Age',
 'Cholesterol',
 'BloodPressure',
 'HeartRate',
 'BMI',
 'MaxHeartRate',
 'ST_Depression',
 'NumberOfMajorVessels'])

Using the function we created (the "calculate_iqr_bounds(df, columns)" function) we managed to find the upper and lower bounds for each numerical column

The next step is to Treat the Outliers. And when preforming this step you get to choose between three options
1. Cap Values: Replacing outliers with bound values
2. Remove: Droping the outlier rows
3. Keep: if outliers are valid
Based on our dataset we will choose to Cap Values

There is one important operation needs to be done before actually treating outliers. That is to determin whether outliers are errors or valid data. To do that we will preform a statistical investigation that is, we will use distribution visualization to. particularly we will use boxplot which will help us to display median, quartiles, and any points that fall outside the typical rang. 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#Plotting a boxplot

sns.set_style("whitegrid")
plt.figure(figsize=(12,6))

sns.boxplot(data=df[numerical_cols])
plt.xticks(rotation=45)
plt.title("Boxplot of Numerical Features")

In [ ]:
#Plotting ST_Depression and NumberOfMajorVessels for better readablity
sns.boxplot(data=df[["ST_Depression", "NumberOfMajorVessels"]])
plt.xticks(rotation=45)
plt.title("ST_Depression and NumberOfMajorVessels")

These tow visualizations indecates that no extreme outliers are present for our numerical datasets. Thus, we can move to the next step.

Resolve Inconsistencies: typos, mismatched categories, or contradicoty data(e.g., age = 5, smoker=yes) which reduces the quality of the data

In [ ]:
df[categorics_cols].dtypes

In [ ]:
df["Gender"].value_counts()#Spotting issues(e.g., Meal instade of Male)

#Incase of a typos
df["Gender"].replace({'Meal': 'Male'})

df[(df["Smoker"] == "yes") & (df["Age"] < 10)]#This checks if a smoker is actually under the age of 10. In that case we would either correct or drop these values


These operations are done repeatedly on all the categorical columns
Fortunately our dataset has no any Inconsistency that would need to be resolved.  

At this stage, we can happily say that our data set has been cleand and fully ready to be analyzed and extract insights out of it to the best levels.
Here is a recap of what we have done in this process of cleaning our dataset
1. Handle Missing Values
2. Remove duplicates (no in our case)
3. Fix data types
4. Detect and treate outliers (only detected them we have not had to treat them
5. Check for Inconsistencies

# Exploratory Data Analysis (EDA)

After, data cleaning we move to the second essential data analysis step which is Exploratory Data Analysis (EDA).
In this step we will go through different phases which are:
1. Understand dataset structure
2. Visualize data Distributions
3. Identify relationships and correlations
4. Analyze target variable
5. Check data quality and pattrens
6. Documents and summerize findings


Before actually jumping into EDA. We need to highlight some important domain knowledge. Having domain knowledge will help in interpreting the dataset's variables, validating our findings and will ensure our analysis aligns with medical realities.

*Heart-Attack*
occures to an individual when flow of blood to the heart is severely reduced or even blocked.
source: [Heart-attack](http://www.mayoclinic.org/diseases-conditions/heart-attack/symptoms-causes/syc-20373106)

**Risk-Factors**
1. **Gender**:Research shows that along with typical risk factors—such as high blood pressure, high cholesterol, and diabetes—men are at increased risk of heart attack earlier in life due to hormone changes, abdominal obesity, and emotional challenges in middle adulthood. source: [Gender](https://www.medstarhealth.org/blog/heart-disease-men#:~:text=Research%20shows%20that%20along%20with%20typical%20risk%20factors%E2%80%94such,abdominal%20obesity%2C%20and%20emotional%20challenges%20in%20middle%20adulthood.)
2. **Cholesterol**: HDL("good" Cholesterol) is the one that needs to be high (idealy above 60). LDL("bad" Cholesterol) the one that needs to be low (below 100). Total should be below 200. source: [Ideal Cholesterol Levels](https://my.clevelandclinic.org/health/articles/11920-cholesterol-numbers-what-do-they-mean)
3. **Blood Pressure**: Blood pressure is considerd high in case of consistent systolic readings of 130 mm Hg or higher, or diastolic readings of 80 mm Hg or higher. source: [Blood Pressure Ranges](https://www.nhlbi.nih.gov/health/high-blood-pressure#:~:text=Your%20blood%20pressure%20is%20high%20when%20you%20have,usually%20occur%20until%20it%20causes%20serious%20health%20problems.)
4. **Heart Rate**: normal heart rate is between 60 and 100 beats per minute (60-100/min). source:[Heart Rates](https://www.healthdirect.gov.au/resting-heart-rate#:~:text=For%20adults%2C%20a%20normal%20resting%20heart%20rate%20is,neck%20and%20counting%20the%20beats%20for%20one%20minute.)
5. **BMI(Body Mass Index)**: A BMI between 18.5 and 25kg/m^2 indicates a normal weight. A BMI of less than 18.5kg/m^2 considered underweight. A BMI between 25kg/m^2 and 29.9kg/m^2 is considered overweight. A BMI of 30kg/m^2 or higher is considered obese. source: [BMI Ranges](https://www.heart.org/en/healthy-living/healthy-eating/losing-weight/bmi-in-adults#:~:text=A%20BMI%20between%2018.5%20and%2025%20kg%2Fm%C2%B2%20indicates,of%2030%20kg%2Fm%C2%B2%20or%20higher%20is%20considered%20obese.)
6. **Smoking**: a major risk factor for hearrt disease. It dameges the blood vassels. source: [Smoking Effects on blood vassels](https://www.nhlbi.nih.gov/health/heart/smoking#:~:text=Smoking%20is%20a%20major%20risk%20factor%20for%20heart,develop%20atherosclerosis%2C%20or%20plaque%20buildup%20in%20the%20arteries.)
7. **Alcoholic Consumption**:Moderate drinking does not reduce the risk of death. Instead, it may increase the risk of chronic diseases like cancer and heart disease. source: [Alcoholic Consumption](https://www.cdc.gov/alcohol/about-alcohol-use/moderate-alcohol-use.html#:~:text=Moderate%20drinking%20increases%20health%20risks%20compared%20to%20not,cancer%20and%20heart%20disease.%203%204%205%20)
8. **Physical Activity**: all adults shoud undertake 150-300mins of moderate-intensity, or 75-150mins of vigorous-intensity per week. source:[Physical Activity](https://pmc.ncbi.nlm.nih.gov/articles/PMC7719906/#:~:text=All%20adults%20should%20undertake%20150%E2%80%93300%20min%20of%20moderate-intensity%2C,moderate-intensity%20and%20vigorous-intensity%20aerobic%20physical%20activity%2C%20per%20week.)
9. **Family History**: family history of cardiovascular disease (CVD) modifies future CVD risk. Siblings of patients with CVD have about 40% risk increase. source: [Family history with CVD](https://pmc.ncbi.nlm.nih.gov/articles/PMC4229162/#:~:text=Family%20history%20of%20CVD%20modifies%20future%20CVD%20risk,CVD%20have%20a%2060%25%20to%2075%25%20risk%20increase.)
10. **Income and Education Level**: Lower income and education correlate with higher risk due to limited access to healthcare. source:[Low Education and CVD](https://pmc.ncbi.nlm.nih.gov/articles/PMC11017042/)
11. **Employment and Residence Status**: Prevalences of adverse health outcomes increased with unemployment duration and were highest for those unable to work. Employment is a health equity issue. source: [Employment and Health issues](https://pmc.ncbi.nlm.nih.gov/articles/PMC8678322/)
12. **Marital Status**: Unmarried patients with known or suspected CAD have an increased risk of all-cause and CVD mortality compared with married ones. source:[Marital Status and CVD](https://www.ahajournals.org/doi/full/10.1161/jaha.117.005890)

In [ ]:
df.describe()

All the statical measurment we would want to consider in order to understand the distribution of our datasets variables.

# EDA 
# 1. Understanding Data Structure

In this section of the EDA, we will try to understand how our data is represented using the essential backbone information of statistics which lies under mean, median, min, max, and quartiles.
A part form these theoretical statical information, we also believe that a picture is worth a thousand words, and that is why we will use a totally general visualization to see the full picture of the dataset. After that we will dive into more details and extract useful analysis.

In [ ]:
df.info()

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)



'Age',
 'Cholesterol',
 'BloodPressure',
 'HeartRate',
 'BMI',
 'MaxHeartRate',
 'ST_Depression',
 'NumberOfMajorVessels'

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(data=df, x="Age", hue="Outcome",bins=100, kde=True)
plt.title("Distribution of Age")
plt.xlabel("Age")
plt.ylabel("Count")
plt.show()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(data=df, y='Cholesterol')
plt.ylabel("Cholesterol (mg/dL")
plt.title("Boxplot of Cholesterol Levels")

In [ ]:
#Comparing cholesterol Levels by gender
plt.figure(figsize=(8,6))
sns.boxplot(data=df, x="Gender", y="Cholesterol")
plt.title("Cholesterol Levels By Gender")
plt.xlabel("Gender")
plt.ylabel("Cholesterol Level (mg/dL)")

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(data=df, x="Gender", hue="Outcome", palette="deep")
plt.title("Distribution of Gender in Heart Attack dataset")
plt.xlabel("Gender")
plt.ylabel("Count")

In [ ]:
df["Gender"].value_counts()

In [ ]:
plt.figure(figsize=(10,6))
correlation_matrix = df[numerical_cols].corr()
correlation_matrix.fillna(0, inplace=True)  # Replace NaN with 0
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap For the Numerical Values")

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
plt.figure(figsize=(8,6))
sns.violinplot(data=df, x="Gender", y="Age")
plt.title("Age Distribution By Gender")

This output suggests that the number of ages for both Genders varies between 30(lower bound) and 80(upper bound)

In [ ]:
plt.figure(figsize=(8,6))


In [ ]:
#Which age has the biggest number of heart attack?
import matplotlib.pyplot as plt

plt.figure(figsize=(18,6))
sns.kdeplot(data=df, x="Age", hue="Outcome", common_norm=False, fill=True, alpha=0.4)
plt.title("Number of Heart Attack By Age")

In [ ]:
df_age = df.groupby(["Age", "Outcome"]).size().reset_index(name="Count")
df_age #a count of unique occurrences by age and outcome columns and resets the index

In [ ]:
#Calculating percentage for each age group 
df_age["Percentage"] = df_age.groupby("Age")["Count"].apply(lambda x: x / x.sum() * 100).reset_index(drop=True)
df_age

In [ ]:
plt.figure(figsize=(14,6))
sns.barplot(
    data=df_age,
    x="Age",
    y="Percentage",
    hue="Outcome"
)
plt.title("Percentage of Heart Attacks by Age")

The number of cases appears to be in a uniform distribution across all ages. The number of cases of Heart-Attack and No-Heart-Attack are similler and almost exactly 50% divided

In [ ]:
df["Smoker"].value_counts()


**At this point can confidently say that the dataset is actually AI-Generated dataset. And all credits, and thanks is actually for the powerful process of Exploratory Data Analysis (EDA) that has revealed the unnatural distributions, highly uniform patterns, and unrealistic correlations. these anomalies highlight the importance of data validation before analysis ensure meaningful insights. In case of starting without preforming any EDA, we would go to directly preform feature Selection and Engineering. In which we would identify most relevant features for the analysis which are the gender, Physical Activity, Gender, and some more. But these features are only considered in real-life aspects (our data is AI-Generated so nothing would have seemed to add-up).**